# 📚 Week 13: Neural Networks - MLP, PyTorch & Keras

---

## 🎯 Learning Objectives

| Day | Topic | Key Concepts |
|-----|-------|--------------|
| 1 | NN Fundamentals | Neurons, activation functions, forward propagation |
| 2 | Backpropagation | Chain rule, gradient descent, learning rate |
| 3 | MLP Architecture | Hidden layers, universal approximation |
| 4 | PyTorch Basics | Tensors, autograd, nn.Module |
| 5 | Keras/TensorFlow | Sequential API, Functional API |
| 6 | Training Practices | Batch size, epochs, callbacks, regularization |
| 7 | Interview Review | Common questions and answers |

---

In [ ]:
# ============================================================
# IMPORTS & SETUP
# ============================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

try:
    import tensorflow as tf
    from tensorflow import keras
    HAS_TF = True
except ImportError:
    HAS_TF = False

plt.style.use('seaborn-v0_8-whitegrid')
print("📥 PyTorch version:", torch.__version__)
if HAS_TF:
    print("📥 TensorFlow version:", tf.__version__)

---

# 📊 Day 1: Neural Network Fundamentals

## The Artificial Neuron

A neuron computes:
$$y = \sigma\left(\sum_{i=1}^{n} w_i x_i + b\right) = \sigma(w^T x + b)$$

Where:
- $x$: Input vector
- $w$: Weight vector
- $b$: Bias
- $\sigma$: Activation function

## Common Activation Functions

| Function | Formula | Range | Use Case |
|----------|---------|-------|----------|
| Sigmoid | $\sigma(z) = \frac{1}{1+e^{-z}}$ | (0, 1) | Output layer (binary) |
| Tanh | $\tanh(z) = \frac{e^z - e^{-z}}{e^z + e^{-z}}$ | (-1, 1) | Hidden layers |
| ReLU | $\max(0, z)$ | [0, ∞) | Hidden layers (most common) |
| Leaky ReLU | $\max(\alpha z, z)$ | (-∞, ∞) | Avoids dead neurons |

## Forward Propagation

For a network with L layers:
$$a^{[l]} = \sigma(W^{[l]} a^{[l-1]} + b^{[l]})$$

---

In [ ]:
# ============================================================
# DAY 1: ACTIVATION FUNCTIONS VISUALIZATION
# ============================================================

z = np.linspace(-5, 5, 100)

# Activation functions
sigmoid = 1 / (1 + np.exp(-z))
tanh = np.tanh(z)
relu = np.maximum(0, z)
leaky_relu = np.where(z > 0, z, 0.1 * z)

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

axes[0, 0].plot(z, sigmoid, 'b-', linewidth=2)
axes[0, 0].axhline(y=0.5, color='r', linestyle='--', alpha=0.5)
axes[0, 0].set_title('Sigmoid: σ(z) = 1/(1+e^-z)', fontweight='bold')
axes[0, 0].set_xlabel('z')
axes[0, 0].set_ylabel('σ(z)')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(z, tanh, 'g-', linewidth=2)
axes[0, 1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[0, 1].set_title('Tanh: (e^z - e^-z)/(e^z + e^-z)', fontweight='bold')
axes[0, 1].set_xlabel('z')
axes[0, 1].set_ylabel('tanh(z)')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(z, relu, 'orange', linewidth=2)
axes[1, 0].set_title('ReLU: max(0, z)', fontweight='bold')
axes[1, 0].set_xlabel('z')
axes[1, 0].set_ylabel('ReLU(z)')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(z, leaky_relu, 'purple', linewidth=2)
axes[1, 1].set_title('Leaky ReLU: max(0.1z, z)', fontweight='bold')
axes[1, 1].set_xlabel('z')
axes[1, 1].set_ylabel('LeakyReLU(z)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

# 📊 Day 2: Backpropagation

## Chain Rule for Gradients

For a loss function $L$ with respect to weights $W^{[l]}$:

$$\frac{\partial L}{\partial W^{[l]}} = \frac{\partial L}{\partial a^{[L]}} \cdot \frac{\partial a^{[L]}}{\partial a^{[L-1]}} \cdots \frac{\partial a^{[l]}}{\partial W^{[l]}}$$

## Gradient Descent Update

$$W := W - \alpha \frac{\partial L}{\partial W}$$

Where $\alpha$ is the learning rate.

## Vanishing/Exploding Gradients

- **Vanishing:** Gradients become too small (sigmoid/tanh in deep networks)
- **Exploding:** Gradients become too large (improper initialization)

**Solutions:**
- ReLU activation
- Proper weight initialization (Xavier, He)
- Batch normalization
- Gradient clipping

---

In [ ]:
# ============================================================
# DAY 2: SIMPLE BACKPROPAGATION EXAMPLE
# ============================================================

class SimpleNN:
    """Simple 2-layer neural network from scratch."""
    
    def __init__(self, input_size, hidden_size, output_size):
        # Xavier initialization
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2.0 / input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2.0 / hidden_size)
        self.b2 = np.zeros((1, output_size))
    
    def relu(self, z):
        return np.maximum(0, z)
    
    def relu_derivative(self, z):
        return (z > 0).astype(float)
    
    def forward(self, X):
        self.z1 = X @ self.W1 + self.b1
        self.a1 = self.relu(self.z1)
        self.z2 = self.a1 @ self.W2 + self.b2
        self.a2 = self.z2  # Linear output for regression
        return self.a2
    
    def backward(self, X, y, learning_rate=0.01):
        m = X.shape[0]
        
        # Output layer gradient
        dz2 = self.a2 - y
        dW2 = (1/m) * self.a1.T @ dz2
        db2 = (1/m) * np.sum(dz2, axis=0, keepdims=True)
        
        # Hidden layer gradient
        da1 = dz2 @ self.W2.T
        dz1 = da1 * self.relu_derivative(self.z1)
        dW1 = (1/m) * X.T @ dz1
        db1 = (1/m) * np.sum(dz1, axis=0, keepdims=True)
        
        # Update weights
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        
        return np.mean((self.a2 - y)**2)  # MSE loss

# Demo: Learn XOR
np.random.seed(42)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

nn = SimpleNN(2, 4, 1)
losses = []

for epoch in range(1000):
    nn.forward(X)
    loss = nn.backward(X, y, learning_rate=0.5)
    losses.append(loss)

print("📊 XOR Learning (From Scratch)")
print("=" * 40)
print(f"Final Loss: {losses[-1]:.6f}")
print(f"\nPredictions:")
for xi, yi in zip(X, nn.forward(X)):
    print(f"   {xi} → {yi[0]:.3f}")

plt.figure(figsize=(8, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Backpropagation Training Loss', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.show()

---

# 📊 Day 3: MLP Architecture

## Multi-Layer Perceptron

$$\text{MLP}: \mathbb{R}^n \rightarrow \mathbb{R}^m$$

- **Input Layer:** Features
- **Hidden Layers:** Non-linear transformations
- **Output Layer:** Predictions

## Universal Approximation Theorem

A feedforward network with a single hidden layer containing a finite number of neurons can approximate any continuous function on compact subsets of $\mathbb{R}^n$.

## Architecture Design

| Problem | Output Activation | Loss Function |
|---------|-------------------|---------------|
| Regression | Linear | MSE |
| Binary Classification | Sigmoid | Binary Cross-Entropy |
| Multi-class | Softmax | Categorical Cross-Entropy |

---

In [ ]:
# ============================================================
# DAY 3: MLP ARCHITECTURE VISUALIZATION
# ============================================================

def draw_neural_net(ax, layer_sizes):
    """Draw a neural network diagram."""
    n_layers = len(layer_sizes)
    v_spacing = 1.0 / (max(layer_sizes) + 1)
    h_spacing = 1.0 / (n_layers + 1)
    
    # Draw neurons
    for i, n_neurons in enumerate(layer_sizes):
        layer_top = v_spacing * (n_neurons - 1) / 2 + 0.5
        for j in range(n_neurons):
            circle = plt.Circle((h_spacing * (i + 1), layer_top - j * v_spacing),
                               v_spacing / 4, color='steelblue', ec='black', zorder=4)
            ax.add_artist(circle)
            
            # Draw connections to next layer
            if i < n_layers - 1:
                next_layer_top = v_spacing * (layer_sizes[i+1] - 1) / 2 + 0.5
                for k in range(layer_sizes[i+1]):
                    line = plt.Line2D([h_spacing * (i + 1), h_spacing * (i + 2)],
                                      [layer_top - j * v_spacing, next_layer_top - k * v_spacing],
                                      c='gray', alpha=0.3)
                    ax.add_artist(line)
    
    # Labels
    labels = ['Input\n(Features)', 'Hidden 1', 'Hidden 2', 'Output\n(Prediction)']
    for i, label in enumerate(labels[:len(layer_sizes)]):
        ax.text(h_spacing * (i + 1), 0.05, label, ha='center', fontsize=10)
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')

fig, ax = plt.subplots(figsize=(12, 6))
draw_neural_net(ax, [4, 8, 6, 1])
ax.set_title('MLP Architecture: 4 → 8 → 6 → 1', fontsize=14, fontweight='bold')
plt.show()

---

# 📊 Day 4: PyTorch Basics

## Core Concepts

1. **Tensors:** Multi-dimensional arrays with GPU support
2. **Autograd:** Automatic differentiation
3. **nn.Module:** Base class for neural networks
4. **Optimizers:** SGD, Adam, etc.

## PyTorch Workflow

```python
# 1. Define model
model = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, output_size)
)

# 2. Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 3. Training loop
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
```

---

In [ ]:
# ============================================================
# DAY 4: PYTORCH NEURAL NETWORK
# ============================================================

# Get financial data
ticker = 'SPY'
end_date = datetime.now()
start_date = end_date - timedelta(days=756)

data = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=True)
returns = data['Close'].pct_change().dropna()

# Create features
df = pd.DataFrame(index=returns.index)
df['ret'] = returns
for lag in [1, 2, 3, 5, 10]:
    df[f'lag_{lag}'] = returns.shift(lag)
df['vol_10'] = returns.rolling(10).std()
df['target'] = returns.shift(-1)
df = df.dropna()

# Prepare PyTorch data
X = df.drop('target', axis=1).values
y = df['target'].values.reshape(-1, 1)

# Train/test split
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Standardize
X_mean, X_std = X_train.mean(axis=0), X_train.std(axis=0)
X_train = (X_train - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

# Convert to tensors
X_train_t = torch.FloatTensor(X_train)
y_train_t = torch.FloatTensor(y_train)
X_test_t = torch.FloatTensor(X_test)
y_test_t = torch.FloatTensor(y_test)

# Define MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLP, self).__init__()
        layers = []
        prev_size = input_size
        for h in hidden_sizes:
            layers.append(nn.Linear(prev_size, h))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))
            prev_size = h
        layers.append(nn.Linear(prev_size, output_size))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# Train model
torch.manual_seed(42)
model = MLP(X_train.shape[1], [32, 16], 1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_losses = []
test_losses = []

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    output = model(X_train_t)
    loss = criterion(output, y_train_t)
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())
    
    model.eval()
    with torch.no_grad():
        test_output = model(X_test_t)
        test_loss = criterion(test_output, y_test_t)
        test_losses.append(test_loss.item())

print("📊 PyTorch MLP Results")
print("=" * 50)
print(f"Final Train Loss: {train_losses[-1]:.6f}")
print(f"Final Test Loss: {test_losses[-1]:.6f}")

# Visualize
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(train_losses, label='Train Loss', alpha=0.8)
ax.plot(test_losses, label='Test Loss', alpha=0.8)
ax.set_xlabel('Epoch')
ax.set_ylabel('MSE Loss')
ax.set_title('PyTorch MLP Training', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

---

# 📊 Day 5: Keras/TensorFlow Basics

## Keras API Styles

**1. Sequential API (Simple):**
```python
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(n_features,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
])
```

**2. Functional API (Complex):**
```python
inputs = keras.Input(shape=(n_features,))
x = keras.layers.Dense(64, activation='relu')(inputs)
x = keras.layers.Dense(32, activation='relu')(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
```

---

In [ ]:
# ============================================================
# DAY 5: KERAS NEURAL NETWORK
# ============================================================

if HAS_TF:
    # Build Keras model
    keras_model = keras.Sequential([
        keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1)
    ])
    
    keras_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    # Train
    history = keras_model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=100,
        batch_size=32,
        verbose=0
    )
    
    print("📊 Keras MLP Results")
    print("=" * 50)
    print(f"Final Train Loss: {history.history['loss'][-1]:.6f}")
    print(f"Final Val Loss: {history.history['val_loss'][-1]:.6f}")
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(history.history['loss'], label='Train Loss')
    ax.plot(history.history['val_loss'], label='Validation Loss')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('MSE Loss')
    ax.set_title('Keras MLP Training', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.show()
else:
    print("⚠️ TensorFlow/Keras not installed. Skipping...")

---

# 📊 Day 6: Training Best Practices

## Hyperparameters

| Parameter | Typical Values | Effect |
|-----------|----------------|--------|
| Learning Rate | 0.001, 0.01 | Speed vs stability |
| Batch Size | 32, 64, 128 | Noise vs speed |
| Hidden Units | 32, 64, 128 | Capacity |
| Dropout | 0.1-0.5 | Regularization |

## Regularization Techniques

1. **L2 Regularization:** Weight decay
2. **Dropout:** Random neuron deactivation
3. **Early Stopping:** Stop when validation loss increases
4. **Batch Normalization:** Normalize layer inputs

## Callbacks

- **EarlyStopping:** Prevent overfitting
- **ReduceLROnPlateau:** Adaptive learning rate
- **ModelCheckpoint:** Save best model

---

In [ ]:
# ============================================================
# DAY 6: TRAINING WITH CALLBACKS (KERAS)
# ============================================================

if HAS_TF:
    # Model with regularization
    model_reg = keras.Sequential([
        keras.layers.Dense(64, activation='relu', 
                          kernel_regularizer=keras.regularizers.l2(0.01),
                          input_shape=(X_train.shape[1],)),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(32, activation='relu',
                          kernel_regularizer=keras.regularizers.l2(0.01)),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1)
    ])
    
    model_reg.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                      loss='mse')
    
    # Callbacks
    callbacks = [
        keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
    ]
    
    history_reg = model_reg.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=200,
        batch_size=32,
        callbacks=callbacks,
        verbose=0
    )
    
    print("📊 Regularized Model with Callbacks")
    print("=" * 50)
    print(f"Stopped at epoch: {len(history_reg.history['loss'])}")
    print(f"Best Val Loss: {min(history_reg.history['val_loss']):.6f}")

---

# 📊 Day 7: Interview Review

## Common Questions

**Q1: What is the vanishing gradient problem?**
> Gradients become very small in deep networks with sigmoid/tanh, making early layers learn slowly. Solutions: ReLU, batch normalization, residual connections.

**Q2: Explain backpropagation.**
> Algorithm to compute gradients using chain rule, propagating errors backward from output to input, updating weights via gradient descent.

**Q3: When to use dropout vs L2 regularization?**
> - **Dropout:** Better for large networks, prevents co-adaptation
> - **L2:** Weight decay, smoother solution, always applicable

**Q4: How to choose learning rate?**
> - Start with 0.001 for Adam
> - Use learning rate schedulers
> - Learning rate finder technique

**Q5: Batch size tradeoffs?**
> - **Small:** More noise, better generalization, slower
> - **Large:** Faster, may overfit, needs higher LR

---

## Summary

| Concept | Key Point |
|---------|-----------|
| Activation | ReLU for hidden, sigmoid/softmax for output |
| Backprop | Chain rule + gradient descent |
| Regularization | Dropout + L2 + Early Stopping |
| PyTorch | Research, flexibility |
| Keras | Production, simplicity |

---